# Projeto 1 - Ciência dos Dados

Nome: Barbara Martins Damasceno

Nome: Daniel Delattre

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [13]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [14]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/daniel/Insper/cdados/cdadosproj1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [15]:
dados = pd.read_excel('MMMercado.xlsx')


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [46]:
import re 
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@//|]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

In [47]:
#Classificando como categoria
dados.loc[:,'relevância'] = dados['relevância'].astype('category')
#Níveis da categoria
dados.relevância.cat.categories = ['Irrelevante', 'Relevante']


dados.head(5)

,Treinamento,relevância
0,nunca vi um mercado tão cheio,Irrelevante
1,rt @folha: guedes diz que vai acionar stf cont...,Relevante
2,deixando tudo mais barato:\ncupom ifood (app) ...,Irrelevante
3,rt @reporterlacerda: inovação no grenal\n\npre...,Irrelevante
4,@ericat_lol mercado a vista vai ficar parado p...,Relevante


In [48]:
#Dividindo a porcentagem em relevante e irrelevante
base = dados.relevância.value_counts(True, sort=False)
irrelevante = base[0]
relevante = base[1]
print(f"Probabilidade de ser relevante: {relevante}")
print(f"Probabilidade de ser irrelevante: {irrelevante}")


Probabilidade de ser relevante: 0.59
Probabilidade de ser irrelevante: 0.41


Criando o classificar

In [73]:
#Novas folhas de dados craidas a partir da relevancia
dados_relev = dados[dados.relevância == 'Relevante'].copy()
dados_irrelev = dados[dados.relevância == 'Irrelevante'].copy()

import csv
frases_ttrelev = dados_relev.Treinamento.tolist()
clean_relev = []

    
output=open('relev.txt','w')
for el in frases_ttrelev:
    output.write(el)
    
for i in frases_ttrelev:
    palavra = cleanup(i)
    
print(frases_ttrelev[0:5])
    


    

['rt @folha: guedes diz que vai acionar stf contra votação que eleva gasto em r$ 20 bilhões https://t.co/kqpnqaxfv3', '@ericat_lol mercado a vista vai ficar parado por 1h agora. depois volta. mas se atingir queda de 20% vai parar e não tem previsão de volta.', 'cabe a mão invisível do mercado pagar meu almoço e me fazer carinho', 'circuit breaker acionado pela terceira vez na semana e dólar acima de r$ 5.00. pelo visto, além do donald trump, o pessoal do mercado financeiro também não leu o artigo do bernardo küster.', 'rt @bitencourtbas: guedes chamou o mercado para brigar nos 5 (dólar). mercado socou r$ na frente dele e fez o que ele dizia querer. \n\ninfel…']


In [74]:
with open("relev.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    relev_raw = arquivo_texto.read()
    
print(relev_raw[0:1000])

rt @folha: guedes diz que vai acionar stf contra votação que eleva gasto em r$ 20 bilhões https://t.co/kqpnqaxfv3@ericat_lol mercado a vista vai ficar parado por 1h agora. depois volta. mas se atingir queda de 20% vai parar e não tem previsão de volta.cabe a mão invisível do mercado pagar meu almoço e me fazer carinhocircuit breaker acionado pela terceira vez na semana e dólar acima de r$ 5.00. pelo visto, além do donald trump, o pessoal do mercado financeiro também não leu o artigo do bernardo küster.rt @bitencourtbas: guedes chamou o mercado para brigar nos 5 (dólar). mercado socou r$ na frente dele e fez o que ele dizia querer. 

infel…rt @alessandromolon: o governo se recusa a reconhecer a gravidade do coronavírus. momentos de seriedade exigem autoridades responsáveis e m…pronto , pessoal mais novo no mercado que está fazendo curso intensivo de cb já tem história pra contar !!!!!rt @farialimaelevat: estagiários filhos do bullmarket, bem vindos ao mercado financeiro!na verdade o mer

### CLEANUP 

In [77]:
relev_c = cleanup(relev_raw.lower())
print(relev_c[:1000])

rt  folha  guedes diz que vai acionar stf contra votação que eleva gasto em r  20 bilhões https   t co kqpnqaxfv3 ericat_lol mercado a vista vai ficar parado por 1h agora  depois volta  mas se atingir queda de 20  vai parar e não tem previsão de volta cabe a mão invisível do mercado pagar meu almoço e me fazer carinhocircuit breaker acionado pela terceira vez na semana e dólar acima de r  5 00  pelo visto  além do donald trump  o pessoal do mercado financeiro também não leu o artigo do bernardo küster rt  bitencourtbas  guedes chamou o mercado para brigar nos 5  dólar   mercado socou r  na frente dele e fez o que ele dizia querer  

infel…rt  alessandromolon  o governo se recusa a reconhecer a gravidade do coronavírus  momentos de seriedade exigem autoridades responsáveis e m…pronto   pessoal mais novo no mercado que está fazendo curso intensivo de cb já tem história pra contar      rt  farialimaelevat  estagiários filhos do bullmarket  bem vindos ao mercado financeiro na verdade o mer

### SPLITZADA

In [79]:
relevzada = relev_c.split()
relevzada[:10]

['rt',
 'folha',
 'guedes',
 'diz',
 'que',
 'vai',
 'acionar',
 'stf',
 'contra',
 'votação']

In [84]:
serie_relev = pd.Series(relevzada)
tabela_relev = serie_relev.value_counts(True)
tabela_relev


o           0.040789
mercado     0.033764
de          0.030818
do          0.024473
que         0.022207
              ...   
elecefab    0.000227
ontem       0.000227
este        0.000227
pede        0.000227
sempre      0.000227
Length: 1510, dtype: float64

### IRRELEVANTE

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**